# IMP - Old Photo Restoration Quick Start

This notebook demonstrates how to use the IMP (Image Restoration Project) system to restore old photos using deep learning.

**Features:**
- Automatic noise removal (denoising)
- Super-resolution upscaling (2x or 4x)
- GPU-accelerated processing
- Easy-to-use pipeline

**Requirements:**
- Google Colab with GPU runtime (Runtime → Change runtime type → GPU)

## 1. Setup and GPU Check

In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✓ GPU Available: {gpu_name}")
    print(f"✓ GPU Memory: {gpu_memory:.2f} GB")
else:
    print("⚠ No GPU detected. Please enable GPU:")
    print("  Runtime → Change runtime type → Hardware accelerator → GPU")

## 2. Clone Repository

In [ ]:
# Clone the IMP repository
import os

if not os.path.exists('imp'):
    print("Cloning IMP repository...")
    !git clone https://github.com/ptlap/imp.git
    print("✓ Repository cloned successfully")
else:
    print("✓ Repository already exists")

# Change to project directory
%cd imp

## 3. Install Dependencies

In [ ]:
# Install required packages
print("Installing dependencies...")
!pip install -q -r requirements.txt
print("✓ Dependencies installed successfully")

## 4. Download Model Weights

In [ ]:
# Download Real-ESRGAN model weights
import os
from src.utils.weight_downloader import WeightDownloader

weights_dir = "weights"
os.makedirs(weights_dir, exist_ok=True)

downloader = WeightDownloader()
weights_path = os.path.join(weights_dir, "realesrgan-x4plus.pth")

if not os.path.exists(weights_path):
    print("Downloading Real-ESRGAN weights...")
    success = downloader.download("realesrgan", weights_path)
    if success:
        print("✓ Model weights downloaded successfully")
    else:
        print("✗ Failed to download weights. Please check your internet connection.")
else:
    print("✓ Model weights already exist")

## 5. Import and Initialize Pipeline

In [ ]:
# Import IMP pipeline
from src.pipeline import OldPhotoRestoration
from src.config import Config

# Create configuration
config = Config.default()
config.models.super_resolution.weights_path = weights_path
config.models.super_resolution.scale = 4  # 4x upscaling
config.models.super_resolution.use_fp16 = True  # Use FP16 for memory efficiency
config.processing.checkpoint_enabled = True

# Initialize pipeline
print("Initializing restoration pipeline...")
pipeline = OldPhotoRestoration(config)
print("✓ Pipeline ready")

## 6. Upload Test Image

Upload your old photo that you want to restore.

In [ ]:
# Upload image from your computer
from google.colab import files
import shutil

print("Please select an image file to upload...")
uploaded = files.upload()

# Get the uploaded filename
input_image_path = list(uploaded.keys())[0]
print(f"✓ Uploaded: {input_image_path}")

# Create output directory
os.makedirs("outputs", exist_ok=True)
output_image_path = os.path.join("outputs", f"restored_{input_image_path}")

## 7. Run Restoration

This will process your image through:
1. Preprocessing (resize, normalize)
2. Denoising (remove noise and artifacts)
3. Super-resolution (upscale 4x)

In [ ]:
# Run restoration
import time

print("Starting restoration process...")
print("This may take 10-30 seconds depending on image size.\n")

start_time = time.time()

try:
    restored_image = pipeline.restore(
        image_path=input_image_path,
        output_path=output_image_path,
        resume=True
    )
    
    elapsed_time = time.time() - start_time
    print(f"\n✓ Restoration completed in {elapsed_time:.2f} seconds")
    print(f"✓ Saved to: {output_image_path}")
    
except Exception as e:
    print(f"\n✗ Error during restoration: {e}")
    raise

## 8. Visualize Results

Compare the original and restored images side by side.

In [ ]:
# Display before and after comparison
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Load images
original = Image.open(input_image_path)
restored = Image.open(output_image_path)

# Create side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Original image
axes[0].imshow(original)
axes[0].set_title(f"Original ({original.size[0]}x{original.size[1]})", fontsize=14, fontweight='bold')
axes[0].axis('off')

# Restored image
axes[1].imshow(restored)
axes[1].set_title(f"Restored ({restored.size[0]}x{restored.size[1]})", fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"\nImage size increased from {original.size[0]}x{original.size[1]} to {restored.size[0]}x{restored.size[1]}")
print(f"Scale factor: {restored.size[0] / original.size[0]:.1f}x")

### Zoom In for Detail Comparison

Let's zoom into a specific region to see the quality improvement.

In [ ]:
# Zoom into center region for detail comparison
def get_center_crop(img, crop_size=200):
    """Extract center crop from image"""
    width, height = img.size
    left = (width - crop_size) // 2
    top = (height - crop_size) // 2
    right = left + crop_size
    bottom = top + crop_size
    return img.crop((left, top, right, bottom))

# Get center crops
original_crop = get_center_crop(original, crop_size=200)
restored_crop = get_center_crop(restored, crop_size=800)  # 4x larger due to upscaling

# Display zoomed comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(original_crop)
axes[0].set_title("Original (Center Crop)", fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(restored_crop)
axes[1].set_title("Restored (Center Crop)", fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("Notice the improved detail and reduced noise in the restored version!")

## 9. Download Restored Image

Download the restored image to your computer.

In [ ]:
# Download the restored image
from google.colab import files

print("Downloading restored image...")
files.download(output_image_path)
print("✓ Download complete")

## 10. Batch Processing (Optional)

Process multiple images at once.

In [ ]:
# Upload multiple images
print("Upload multiple images for batch processing...")
uploaded_batch = files.upload()

# Get list of uploaded files
image_paths = list(uploaded_batch.keys())
print(f"\n✓ Uploaded {len(image_paths)} images")

# Process batch
print("\nStarting batch restoration...")
successes, failures = pipeline.batch_restore(
    image_paths=image_paths,
    output_dir="outputs",
    max_retries=2
)

# Print summary
print(f"\n{'='*50}")
print(f"Batch Processing Complete")
print(f"{'='*50}")
print(f"✓ Successful: {len(successes)}")
print(f"✗ Failed: {len(failures)}")

if failures:
    print("\nFailed images:")
    for failure in failures:
        print(f"  - {failure['input_path']}: {failure['error']}")

## 11. Advanced Configuration (Optional)

Customize processing parameters for different results.

In [ ]:
# Create custom configuration
custom_config = Config.default()

# Adjust denoising strength (higher = more aggressive)
custom_config.models.denoising.strength = 15  # Default is 10

# Change upscaling factor
custom_config.models.super_resolution.scale = 2  # 2x instead of 4x (faster)

# Adjust tiling parameters for large images
custom_config.models.super_resolution.tile_size = 512
custom_config.models.super_resolution.tile_overlap = 64

# Disable checkpoints for faster processing
custom_config.processing.checkpoint_enabled = False

# Create new pipeline with custom config
custom_pipeline = OldPhotoRestoration(custom_config)

print("✓ Custom pipeline created with:")
print(f"  - Denoising strength: {custom_config.models.denoising.strength}")
print(f"  - Upscaling factor: {custom_config.models.super_resolution.scale}x")
print(f"  - Checkpoints: {'Enabled' if custom_config.processing.checkpoint_enabled else 'Disabled'}")

## 12. Memory Usage Monitoring

Check GPU memory usage during processing.

In [ ]:
# Check GPU memory usage
from src.utils.memory import MemoryManager

if torch.cuda.is_available():
    usage = MemoryManager.get_memory_usage()
    print("GPU Memory Usage:")
    print(f"  Allocated: {usage['allocated']:.2f} GB")
    print(f"  Reserved: {usage['reserved']:.2f} GB")
    print(f"  Max Allocated: {usage['max_allocated']:.2f} GB")
    
    # Clear cache if needed
    MemoryManager.clear_cache()
    print("\n✓ GPU cache cleared")
else:
    print("No GPU available for memory monitoring")

## Troubleshooting

### Out of Memory Error
If you encounter OOM errors:
1. Reduce the upscaling factor (use 2x instead of 4x)
2. Enable FP16: `config.models.super_resolution.use_fp16 = True`
3. Reduce tile size: `config.models.super_resolution.tile_size = 256`

### Slow Processing
If processing is too slow:
1. Make sure GPU is enabled (check cell 1)
2. Disable checkpoints: `config.processing.checkpoint_enabled = False`
3. Use 2x upscaling instead of 4x

### Model Download Fails
If weight download fails:
1. Check your internet connection
2. Try running the download cell again
3. Manually download from: https://github.com/xinntao/Real-ESRGAN/releases

### Image Quality Issues
If results are not satisfactory:
1. Adjust denoising strength (try values between 5-20)
2. Try different upscaling factors
3. Check if input image is too degraded

## Next Steps

- Experiment with different configuration parameters
- Try batch processing multiple images
- Check out the [documentation](https://github.com/yourusername/imp) for more advanced features
- Report issues or contribute at: https://github.com/yourusername/imp/issues